## Prompt Optimization

[Read the full post](https://www.saxifrage.xyz/prompt-optimization)


In [36]:
# Load in the input text file
with open('texts/input_text.txt', 'r') as file:
    text = file.read()

# Print the text
print(text)

Value-based pricing is a strategy which sets prices primarily, but not exclusively, on the value, perceived or estimated, to the customer rather than on the cost of the product, the market price, competitors' prices, or historical trends. The primary aim of a value-based pricing approach is to break free from cost-plus pricing and focus on the value that a product or service delivers to a customer.

In value-based pricing, a business conducts research to understand how customers value their product, what features are most important to customers, and how they rank against competitive offerings. For example, if a software company finds out that their product saves an average customer 10 hours a month, they might price their product based on a portion of how much that time is worth to the customer. 

The steps in implementing a value-based pricing strategy typically include:

1. **Understanding the Customer**: The company needs to understand the target market, and what they value in a pro

In [37]:
# Load in the reference file we manually wrote
with open('texts/reference.txt', 'r') as file:
    reference = file.read()

# Print the reference text
print(reference)


There are three main ways to set your pricing strategy: cost, market, and value. Value-based pricing is where you base your price based on the estimated value you’re bringing to the business, rather than what competitors are charging, or what it costs you to deliver the service. The main benefit being that you can potentially charge a lot more this way.

In order to execute this pricing strategy, you need to research how customers value your product, what features are most important, and how you might rethink your positioning to move up the value chain. For example, if your software saves customers $10k per month, you can charge say 1/10th of that and the customer will still be happy, even if your software only costs you pennies to run.

Technical founders will often just take the cost and add 10%, which in software severely underprices your offering. Pricing based on market value, i.e. undercutting your main competitor by 10%, can also be a problem, because there’s nothing stopping th

In [38]:
# Set up the first prompt template
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# create a string template for a System role
system_template = """You will provided with the sample text.
Your task is to rewrite the text into a different writing style.
The writing style is: 
- Concise and to the point
- Professional and knowledgeable tone
- Relatable and accessible language
- Uses storytelling to convey ideas
- Relies on examples and case studies
- Mixes personal anecdotes with industry insights
- Provides actionable advice and tips
- Uses subheadings and bullet points for easy readability
"""
# create a prompt template for a System role
system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template)
# create a string template for a Human role
human_template = "{input_text}"
# create a prompt template for a Human role
human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_template)
# create chat prompt template out of one or several message prompt templates
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt_template, human_message_prompt_template])
# generate a final prompt by passing the variables
final_prompt_template = chat_prompt_template.format_prompt(input_text=text)
final_prompt = final_prompt_template.to_messages()

print(final_prompt)

[SystemMessage(content='You will provided with the sample text.\nYour task is to rewrite the text into a different writing style.\nThe writing style is: \n- Concise and to the point\n- Professional and knowledgeable tone\n- Relatable and accessible language\n- Uses storytelling to convey ideas\n- Relies on examples and case studies\n- Mixes personal anecdotes with industry insights\n- Provides actionable advice and tips\n- Uses subheadings and bullet points for easy readability\n', additional_kwargs={}), HumanMessage(content="Value-based pricing is a strategy which sets prices primarily, but not exclusively, on the value, perceived or estimated, to the customer rather than on the cost of the product, the market price, competitors' prices, or historical trends. The primary aim of a value-based pricing approach is to break free from cost-plus pricing and focus on the value that a product or service delivers to a customer.\n\nIn value-based pricing, a business conducts research to underst

In [39]:
# run n completions with a prompt and tags
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.manager import tracing_v2_enabled
import hashlib
from dotenv import load_dotenv

load_dotenv()

# create the llm object
chat = ChatOpenAI(model="gpt-4", temperature=1)

def gen_ex(prompt, test_runs=1):
    # hash to get a unique id that will be the same if passed the same prompt
    prompt_str = ""
    for message in prompt:
        prompt_str += f"{message.type.capitalize()}: {message.content}\n\n"
    prompt_str = prompt_str[:-2] # Remove the last two \n characters

    prompt_id = hashlib.md5(prompt_str.encode()).hexdigest()

    # create an object to store the results
    results = []

    # run n completions with the prompt and tags
    for _ in range(test_runs):
        with tracing_v2_enabled(project_name="content rewriter", tags=[prompt_id]):
            result = chat(prompt)
            results.append(result)

    return results

results = gen_ex(final_prompt, test_runs=1)

In [40]:
print(results[0].content)

**Shaping Prices, Directing Profits: The Art of Value-based Pricing**

Let me tell you a little secret from the business world: there's a method of pricing that isn't bound by the shackles of production costs, market averages, competitor’s figures, or historical trends. It's called value-based pricing, and it's all about accurately assessing your customers’ perceived value of a product or service. 

Think of it like this: if you invented a software that saves its users an average of 10 hours each month, wouldn't it make more sense to price it according to how much that time-saving is worth to your customer, rather than the cost of developing that software? The answer to that might as well be your lightbulb moment.

**The Nitty-Gritty of Value-based Pricing**

To make sure you understand this technique, let me offer a concrete step-by-step guide:

* **Know Thy Customer**: Start by getting under the skin of your target market. Use surveys, trend analysis, competitor studies, or even one-

In [41]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("embedding_distance")

prompt_score = evaluator.evaluate_strings(prediction=results[0].content, reference=reference)['score']
print(f"Prompt score: {prompt_score:.2f}")

Prompt score: 0.08


In [42]:
# generate the results for the test run
results_a = gen_ex(final_prompt, test_runs=20)

In [53]:
# Tally up the scores for the test_runs

import numpy as np
all_scores = []

for i in range(len(results_a)):
    prompt_score = evaluator.evaluate_strings(prediction=results_a[i].content, reference=reference)['score']
    all_scores.append(prompt_score)
    print(f"Run {i+1}: {prompt_score:.3f}")


Run 1: 0.052
Run 2: 0.056
Run 3: 0.062
Run 4: 0.050
Run 5: 0.064
Run 6: 0.077
Run 7: 0.070
Run 8: 0.065
Run 9: 0.051
Run 10: 0.075
Run 11: 0.056
Run 12: 0.064
Run 13: 0.069
Run 14: 0.065
Run 15: 0.090
Run 16: 0.054
Run 17: 0.096
Run 18: 0.066
Run 19: 0.070
Run 20: 0.051


In [44]:
# create new prompt to test
from langchain.prompts import AIMessagePromptTemplate

# import the example from texts/example.txt
with open("texts/example.txt", "r") as f:
    example = f.read()

# create a string template for a System role
system_template_b = """You will provided with the sample text.
Your task is to rewrite the text into a different writing style.
The writing style is: 
- Concise and to the point
- Professional and knowledgeable tone
- Relatable and accessible language
- Uses storytelling to convey ideas
- Relies on examples and case studies
- Mixes personal anecdotes with industry insights
- Provides actionable advice and tips
- Uses subheadings and bullet points for easy readability
"""
# create a prompt template for a System role
system_message_prompt_template_b = SystemMessagePromptTemplate.from_template(
    system_template_b)

# create a string template for a Human role
human_template = "Here is an example of the writing style:\n{example}"
# create a prompt template for a Human role
human_message_prompt_template_b = HumanMessagePromptTemplate.from_template(human_template)

# create a string template for an AI role
ai_template = "Thank you for the example of the writing style. Now please provide the sample text."

# create a prompt template for an AI role
ai_message_prompt_template = AIMessagePromptTemplate.from_template(ai_template)

# create chat prompt template out of one or several message prompt templates
chat_prompt_template_b = ChatPromptTemplate.from_messages(
    [system_message_prompt_template_b, human_message_prompt_template_b, ai_message_prompt_template, human_message_prompt_template]) # human_message_prompt_template defined earlier
# generate a final prompt by passing the variables
final_prompt_template_b = chat_prompt_template_b.format_prompt(input_text=text, example=example)
final_prompt_b = final_prompt_template_b.to_messages()

print(final_prompt_b)

[SystemMessage(content='You will provided with the sample text.\nYour task is to rewrite the text into a different writing style.\nThe writing style is: \n- Concise and to the point\n- Professional and knowledgeable tone\n- Relatable and accessible language\n- Uses storytelling to convey ideas\n- Relies on examples and case studies\n- Mixes personal anecdotes with industry insights\n- Provides actionable advice and tips\n- Uses subheadings and bullet points for easy readability\n', additional_kwargs={}), HumanMessage(content='Here is an example of the writing style:\nHow Agencies Make Money\nThere are three major metrics that determine how much money an agency makes. It’s important to understand each one and how it drives incentives.\n\nRevenue — how much do they charge clients?\nVariable Costs — how much does it cost to service clients?\nFixed Costs — what overheads do they have aside from client work?\nRevenue\nThere are lots of ways to charge for agency services, by the hour, based 

In [45]:
# generate the test_runs for the given prompt
results_b = gen_ex(final_prompt_b, test_runs=20)

In [54]:
# tally up the scores
all_scores_b = []

for i in range(len(results_b)):
    prompt_score = evaluator.evaluate_strings(prediction=results_b[i].content, reference=reference)['score']
    all_scores_b.append(prompt_score)
    print(f"Run {i+1}: {prompt_score:.3f}")

Run 1: 0.063
Run 2: 0.072
Run 3: 0.065
Run 4: 0.055
Run 5: 0.053
Run 6: 0.077
Run 7: 0.076
Run 8: 0.067
Run 9: 0.068
Run 10: 0.072
Run 11: 0.061
Run 12: 0.056
Run 13: 0.065
Run 14: 0.051
Run 15: 0.058
Run 16: 0.059
Run 17: 0.096
Run 18: 0.066
Run 19: 0.057
Run 20: 0.062


In [57]:
# print the average of the vs_baseline scores
print(f"\nAverage A: {np.mean(all_scores):.5f}")
print(f"\nAverage B: {np.mean(all_scores_b):.5f}")


Average A: 0.06509

Average B: 0.06502


In [74]:
import pandas as pd

df = pd.DataFrame({'Prompt A': all_scores, 'Prompt B': all_scores_b}, index=range(1, len(all_scores_b)+1))

# calculate the mean of each column
df.loc['mean'] = df.mean()

# format dataframe to 2 decimal places
df = df.applymap(lambda x: "{0:.3f}".format(x))

df



,Prompt A,Prompt B
1,0.052,0.063
2,0.056,0.072
3,0.062,0.065
4,0.050,0.055
5,0.064,0.053
6,0.077,0.077
7,0.070,0.076
8,0.065,0.067
9,0.051,0.068
10,0.075,0.072
